In [1]:
from utils import *
import tensorflow as tf
from keras import backend as K

2022-10-23 01:02:59.561483: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 01:02:59.672930: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-23 01:03:00.157119: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/moises/workspace/devel/lib
2022-10-23 01:03:00.157169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.

Check if keras is using GPU

In [2]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
K._get_available_gpus()

2022-10-23 01:03:00.655608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 01:03:00.694899: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 01:03:00.733349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 01:03:00.733509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



2022-10-23 01:03:01.113929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 01:03:01.114118: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 01:03:01.114267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 01:03:01.114358: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 199 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
2022-10-23 01:03:01.115761: I tensorflow/stream_executor/cuda/cuda_gpu_executor

['/device:GPU:0']

Get tickers

In [3]:
tickers_sets_path = 'data/tickers/sets/'

sets_files = sorted([tickers_sets_path + x for x in listdir(tickers_sets_path) if 'json' in x], reverse=False)

tickers_sets = []

for set_file in sets_files:
    with open(set_file, 'r') as f:
        tickers_sets.append(json.load(f))

Read dataset

In [4]:
df = pd.read_csv('data/dataset.csv')
df = df.set_index('Date')
# df = df[:-60]

Define training sets and data size

In [5]:
in_sets = [0,1,2]
out_sets = [2]
data_size = 1000

input_tickers = set()
output_tickers = set()
for in_set in in_sets:
    input_tickers = input_tickers.union(set(tickers_sets[in_set]))

for out_set in out_sets:
    output_tickers = output_tickers.union(set(tickers_sets[out_set]))

input_tickers = list(input_tickers)
output_tickers  = list(output_tickers)


Process data

In [6]:
scaler, x, y, dic = dataa(df, data_size, input_tickers, output_tickers, \
                        step_size=1, input_size=40, output_size=20, \
                        feature_range=(0,1), return_log=False)

493 14


Split dataset i training, validation and test

In [7]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x,y)

Define hyperparameters

In [8]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info_list = [
    [
        (75, 0.15, True), 
        (75, 0.15, True),
        (75, 0.15, True),
        (75, 0.15, False)
    ],
    [
        (250, 0.20, True), 
        (250, 0.20, True),
        (250, 0.20, True),
        (250, 0.20, False)
    ],
    [
        (600, 0.15, True),
        (600, 0.15, False)
    ],
    [
        (80, 0.10, True), 
        (80, 0.10, True),
        (80, 0.10, False)
    ],
    [
        (450, 0.15, True), 
        (450, 0.15, True),
        (450, 0.15, False)
    ]
]



Create Models

In [9]:
models = []
hyperparameters = range(len(layers_info_list))

for i in hyperparameters:
    layers_info = layers_info_list[i]
    models.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

Train networks

In [10]:
history_list = []
epochs_list = [125, 125, 125, 125, 125]

set_name = 'set2/'
plot_path_dir = 'metrics/' + set_name
extra_info = ''

j=0
for i in hyperparameters:

    model = models[j]
    epochs = epochs_list[j]
    j += 1
    try:
        
        history = model.fit(
            x=x_train,
            y=y_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x_val, y_val)
        )

        history_list.append(history)

        model_name = 'LSTM' + str(i) + extra_info
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    

Epoch 1/125
ERRO NO MODELO 0
Epoch 1/125


2022-10-23 01:03:09.232362: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2022-10-23 01:03:09.232697: E tensorflow/stream_executor/cuda/cuda_dnn.cc:398] Possibly insufficient driver version: 520.61.5
2022-10-23 01:03:09.232719: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at cudnn_rnn_ops.cc:1554 : UNKNOWN: Fail to find the dnn implementation.
